In [2]:
import os
import io
import time
import pickle
import numpy as np
from numpy.random import choice, shuffle
import tqdm.notebook
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Nadam
from keras.layers import Dense, Activation, Dropout, Input, LeakyReLU
from keras.layers import Input, Embedding, Bidirectional, LSTM, GRU
from keras.models import Model, Sequential
from keras_self_attention import SeqSelfAttention

Using TensorFlow backend.


In [3]:
with open('pickle/all_song.pickle','rb') as read_file:
    all_song= pickle.load(read_file)
with open('pickle/notes_index_dict.pickle','rb') as read_file:
    notes_index_dict= pickle.load(read_file)

In [4]:
chopin_target = []
chopin_train = []

for song in all_song:
    if song['composer'] == 'chopin':
        chopin_train.append(song['train'])
        chopin_target.append(song['target'])

In [5]:
len(chopin_target)

48

In [6]:
target = [y for x in chopin_target for y in x]
train = [y for x in chopin_train for y in x]

In [7]:
len(target)

226800

In [8]:
rnn_unit = 128
dropout = 0.3
optimizer = Nadam()
unique_notes = len(notes_index_dict)
seq_len = 50
dense_unit = 64
output_emb = 100

def create_model(seq_len, unique_notes, dropout, output_emb, rnn_unit, dense_unit):
    inputs = Input(shape=(seq_len,))
    embedding = Embedding(input_dim=unique_notes+1, output_dim=output_emb, input_length=seq_len)(inputs)
    GRU_1 = Bidirectional(GRU(rnn_unit, return_sequences=True))(embedding)
    selfattention_1, att_vector = SeqSelfAttention(
        return_attention=True,
          attention_activation='sigmoid', 
          attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
          attention_width=50, 
          kernel_regularizer=tf.keras.regularizers.l2(1e-4),
          bias_regularizer=tf.keras.regularizers.l1(1e-4),
          attention_regularizer_weight=1e-4)(GRU_1)
    dropout_1 = Dropout(dropout)(selfattention_1)
    GRU_2 = Bidirectional(GRU(rnn_unit, return_sequences=True))(dropout_1)
    selfattention_2, att_vector2 = SeqSelfAttention(
        return_attention=True,
          attention_activation='sigmoid', 
          attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
          attention_width=50, 
          kernel_regularizer=tf.keras.regularizers.l2(1e-4),
          bias_regularizer=tf.keras.regularizers.l1(1e-4),
          attention_regularizer_weight=1e-4)(GRU_2)
    dropout_2 = Dropout(dropout)(selfattention_2)
    GRU_3 = Bidirectional(GRU(rnn_unit))(dropout_2)
    dropout_3 = Dropout(dropout)(GRU_3)
    dense_1 = Dense(dense_unit)(dropout_3)
    leakyrelu = LeakyReLU()(dense_1)
    outputs = Dense(unique_notes+1, activation = 'softmax')(leakyrelu)

    model = Model(inputs = inputs, outputs = outputs, name = 'music_gen')
    return model


In [9]:
chopin = create_model(seq_len, unique_notes, dropout, output_emb, rnn_unit, dense_unit)

In [10]:
chopin.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

In [11]:
chopin.fit(train, target, batch_size=500, epochs = 10)

Epoch 1/10
454/454 [==============================] - 1762s 4s/step - loss: 3.7864
Epoch 2/10
454/454 [==============================] - 1775s 4s/step - loss: 2.2488
Epoch 3/10
454/454 [==============================] - 1767s 4s/step - loss: 1.4566
Epoch 4/10
454/454 [==============================] - 1378s 3s/step - loss: 0.5477
Epoch 5/10
454/454 [==============================] - 874s 2s/step - loss: 0.2799
Epoch 6/10
454/454 [==============================] - 877s 2s/step - loss: 0.1897
Epoch 7/10
454/454 [==============================] - 876s 2s/step - loss: 0.1773
Epoch 8/10
454/454 [==============================] - 879s 2s/step - loss: 0.1388
Epoch 9/10
454/454 [==============================] - 876s 2s/step - loss: 0.1171
Epoch 10/10
454/454 [==============================] - 877s 2s/step - loss: 0.1042


In [14]:
chopin.save('chopin_10ep.h5')

## Train 10 more epochs

In [15]:
chopin.fit(train, target, batch_size=500, epochs = 10)

Epoch 1/10
454/454 [==============================] - 851s 2s/step - loss: 0.1066
Epoch 2/10
454/454 [==============================] - 861s 2s/step - loss: 0.1051
Epoch 3/10
454/454 [==============================] - 865s 2s/step - loss: 0.0824
Epoch 4/10
454/454 [==============================] - 870s 2s/step - loss: 0.0931
Epoch 5/10
454/454 [==============================] - 871s 2s/step - loss: 0.0725
Epoch 6/10
454/454 [==============================] - 873s 2s/step - loss: 0.0797
Epoch 7/10
454/454 [==============================] - 869s 2s/step - loss: 0.0687
Epoch 8/10
454/454 [==============================] - 871s 2s/step - loss: 0.0650
Epoch 9/10
454/454 [==============================] - 868s 2s/step - loss: 0.0615
Epoch 10/10
454/454 [==============================] - 867s 2s/step - loss: 0.0613


In [16]:
chopin.save('chopin_20ep.h5')